In [16]:
# https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=0gZ-l0npPIca

In [ ]:
# pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
# pip install torch-geometric

In [1]:
import os
import json
import time
import random
#import scipy
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sp
from typing import Optional, Tuple
from sklearn.metrics import classification_report

from tqdm import tqdm

import networkx as nx

import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F

import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from torch.nn import Linear
from torch_geometric.nn import GATConv

from torch_geometric.nn import global_mean_pool


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

# with open("file_info.json", 'r') as f :
#     file_info = json.load(f)
    
names= ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

/home/dxlab/anaconda3/envs/jy_gpu_py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH = '/media/dxlab/storage/junyeop/'
#len(os.listdir(PATH + "image_dep")), len(os.listdir(PATH + "image_daily"))

adj_daily = os.listdir(PATH + "adj_mat/daily/")
adj_depression = os.listdir(PATH + "adj_mat/depression/")
adj_diagnosis = os.listdir(PATH + "adj_mat/diagnosis/")


with open(PATH + 'vlog_metadata_diagnosis.pickle', 'rb') as f :
    meta_diagnosis = pickle.load(f)

with open(PATH + 'vlog_metadata_daily.pickle', 'rb') as f :
    meta_daily = pickle.load(f)
with open(PATH + 'vlog_metadata_daily2.pickle', 'rb') as f :
    meta_daily2= pickle.load(f)    
meta_daily.update(meta_daily2)

with open(PATH + 'vlog_backup/vlog_dep_det/vlog_metadata_description.pickle', 'rb') as f :
    meta_depression = pickle.load(f)


file_info = dict()

have_meta_daily = dict()
for i in adj_daily :
    if os.path.splitext(i)[0] in list(meta_daily.keys()) :
        if type(meta_daily[os.path.splitext(i)[0]]) == str :
            continue
        have_meta_daily[os.path.splitext(i)[0]] = meta_daily[os.path.splitext(i)[0]]
        have_meta_daily[os.path.splitext(i)[0]]['label'] = 0
        
# have_meta_depression = dict()
# for i in adj_depression :
#     if os.path.splitext(i)[0] in list(meta_depression.keys()) :
#         if type(meta_depression[os.path.splitext(i)[0]]) == str :
#             continue
#         have_meta_depression[os.path.splitext(i)[0]] = meta_depression[os.path.splitext(i)[0]]
#         have_meta_depression[os.path.splitext(i)[0]]['label'] = 1
        
have_meta_diagnosis = dict()
for i in adj_diagnosis :
    if os.path.splitext(i)[0] in list(meta_diagnosis.keys()) :
        if type(meta_diagnosis[os.path.splitext(i)[0]]) == str :
            continue        
        have_meta_diagnosis[os.path.splitext(i)[0]] = meta_diagnosis[os.path.splitext(i)[0]]
        have_meta_diagnosis[os.path.splitext(i)[0]]['label'] = 1
        
file_info = dict()
file_info.update(have_meta_daily)
#file_info.update(have_meta_depression)
file_info.update(have_meta_diagnosis)
        
len(file_info), len(have_meta_daily), len(have_meta_diagnosis) #len(have_meta_depression), 

(4125, 1888, 2237)

In [3]:
random.seed(0)
file_info = list(file_info.items())
random.shuffle(file_info)
file_info = dict(file_info)

In [4]:
NODE_ATTR = 80
HIDDEN_DIM = 1024
EPOCHS = 500
BATCH_SIZE = 8
LEARNING_RATE = 1e-3

In [5]:
feature_matrix = torch.eye(len(names), dtype = torch.float32) # node feature matrix all one vector

In [6]:
labels = list()
c0, c1, c2 = 0, 0, 0

for x in file_info :
    if file_info[x]['label'] == 0:
        labels.append(0)
        c0 += 1
#     elif file_info[x]['label'] == 1 :
#         labels.append(1)
#         c1 += 1
    else :
        labels.append(1)
        c2 += 1 
labels = torch.LongTensor(labels).unsqueeze(dim = 1)
print("daily vlog : {}\ndiagnosis vlog : {}".format(c0, c2))

daily vlog : 1888
diagnosis vlog : 2237


In [7]:
dataset = list()

for idx, mat_dir in tqdm(enumerate(file_info)) :

    if labels[idx] == 0 :
        with open(PATH + "adj_mat/daily/" + mat_dir + '.npy', 'rb') as f:
            m = np.load(f)
#     elif labels[idx] == 1:
#         with open(PATH + "adj_mat/depression/" + mat_dir + '.npy', 'rb') as f:
#             m = np.load(f)
    else :
        with open(PATH + "adj_mat/diagnosis/" + mat_dir + '.npy', 'rb') as f:
            m = np.load(f)        
    m = nx.adjacency_matrix(nx.from_numpy_array(m))
    edge_index, edge_attr = torch_geometric.utils.from_scipy_sparse_matrix(m)
    data = Data(edge_index=edge_index, x=feature_matrix, edge_attr = edge_attr, y = labels[idx])
    #data = Data(edge_index=edge_index,  edge_attr = edge_attr, y = labels[idx])
    dataset.append(data)
    
random.shuffle(dataset)

4125it [00:03, 1065.82it/s]


In [8]:
train_dataset = dataset[:int(len(dataset)*0.8)]
valid_dataset = dataset[int(len(dataset)*0.8):int(len(dataset)*0.9)]
test_dataset = dataset[int(len(dataset)*0.9):]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
y_train = [x.y for x in train_dataset]
y_valid = [x.y for x in valid_dataset]
y_test = [x.y for x in test_dataset]

In [10]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim: int):
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        score = torch.bmm(query, key.transpose(1, 2)) / self.sqrt_dim

        if mask is not None:
            score.masked_fill_(mask.view(score.size()), -float('Inf'))

        attn = F.softmax(score, -1)
        context = torch.bmm(attn, value)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int = HIDDEN_DIM, num_heads: int = 8):
        super(MultiHeadAttention, self).__init__()

        assert d_model % num_heads == 0, "d_model % num_heads should be zero."

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        self.scaled_dot_attn = ScaledDotProductAttention(self.d_head)
        self.query_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.key_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.value_proj = nn.Linear(d_model, self.d_head * num_heads)

    def forward(self,
                query: Tensor,
                key: Tensor,
                value: Tensor,
                mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        
        batch_size = value.size(0)

        query = self.query_proj(query).view(batch_size, -1, self.num_heads, self.d_head)  # BxQ_LENxNxD
        key = self.key_proj(key).view(batch_size, -1, self.num_heads, self.d_head)      # BxK_LENxNxD
        value = self.value_proj(value).view(batch_size, -1, self.num_heads, self.d_head)  # BxV_LENxNxD

        query = query.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxQ_LENxD
        key = key.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)      # BNxK_LENxD
        value = value.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxV_LENxD

        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.num_heads, 1, 1)  # BxNxQ_LENxK_LEN

        context, attn = self.scaled_dot_attn(query, key, value, mask)

        context = context.view(self.num_heads, batch_size, -1, self.d_head)
        context = context.permute(1, 2, 0, 3).contiguous().view(batch_size, -1, self.num_heads * self.d_head)  # BxTxND

        return context, attn

In [11]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(Model, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(NODE_ATTR, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)        
        
        
        self.lin1 = Linear(hidden_channels, hidden_channels//2)
        self.lin2 = Linear(hidden_channels//2, hidden_channels//4)
        
        self.classifier = Linear(hidden_channels//4, len(labels.unique()))
        
        self.attn = MultiHeadAttention()
    
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)


        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        
        # 3. self Attention layer
        x = self.attn(x, x, x)[0]

        # 5. Apply a final classifier
        x = F.dropout(self.lin1(x), p=0.5, training=self.training)
        x = F.dropout(self.lin2(x), p=0.5, training=self.training)
        x = self.classifier(x)
        return x.squeeze(dim = 1)

In [12]:
def train(opt):
    model.train()
    correct = 0
    total_loss = 0.0
    for data in train_loader:  # Iterate in batches over the training dataset.
        
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)

        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.

        
        loss = criterion(out, data.y).to(device)  # Compute the loss.
        loss.backward()  # Derive gradients.
        total_loss += loss.item()
        
        opt.step()  # Update parameters based on gradients.
        opt.zero_grad()  # Clear gradients.
        
        pred = out.argmax(dim = 1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    
    return correct / len(train_loader.dataset), total_loss / len(train_loader)
    
def test(loader):
    model.eval()

    correct = 0
    total_loss = 0.0  
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)
        
        out = model(data.x, data.edge_index, data.batch)  
        loss = criterion(out, data.y).to(device)  # Compute the loss.
        
        pred = out.argmax(dim = 1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        
        total_loss += loss.item()
        
    return correct/len(loader.dataset), total_loss / len(loader)  # Derive ratio of correct predictions.

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Model(hidden_channels=HIDDEN_DIM).to(device)
#model = GCN(hidden_channels=HIDDEN_DIM).to(device)

In [14]:
LR_LIST = [1e-4, 1e-5, 1e-6]
WEIGHT_DECAY = [1e-3, 1e-4, 1e-5]

grid_result = pd.DataFrame(columns=['epoch', 'learning_rate', 'weight_decay', 
                                    'train_acc', 'valid_acc', 
                                    'train_loss', 'valid_loss'],
                           index= range(len(LR_LIST) * len(WEIGHT_DECAY)))

iter_ = 0

for lr in LR_LIST :
    for wd in WEIGHT_DECAY :
        model = Model(hidden_channels=HIDDEN_DIM).to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        criterion = torch.nn.CrossEntropyLoss()

        train_loss_list = list()
        
        valid_loss_list = list()

        min_valid_loss = 2e+10

        best_epoch = 0
        pbar = tqdm(range(EPOCHS), 
                    ascii = ' =')
        
        for epoch in pbar :
        #for epoch in tqdm(range(EPOCHS)) :
            
            time.sleep(1)
            train_acc, train_loss = train(optimizer)
            valid_acc, valid_loss = test(valid_loader)
            train_loss_list.append(train_loss)
            valid_loss_list.append(valid_loss)
            
            if valid_loss < min_valid_loss :
                best_epoch = epoch
                torch.save(model, "./checkpoint/grid_search/0_binary_depression_unimodal/GAT/best_model_binary_{}_{}.pt".format(lr, wd))
                torch.save(model.state_dict(), './checkpoint/grid_search/0_binary_depression_unimodal/GAT/best_model_binary_parameters_{}_{}.pt'.format(lr, wd))
                min_valid_loss = valid_loss
                grid_result.iloc[iter_, :] = [epoch, lr, wd, 
                                              train_acc, train_loss, 
                                              valid_acc, valid_loss]
                best_ta = train_acc
                best_va = valid_acc
                best_tl = train_loss
                best_vl = valid_loss
                
            else :
                pass
            pbar.set_description(f'Epoch {epoch+1} ')
            pbar.set_postfix({'LR' : lr,
                              'WD' : wd,
                              'TA' : train_acc,
                              'TL' : train_loss, 
                              'VA' : valid_acc,
                              'VL' : valid_loss})        
        print("Epoch : {}, LR : {}, WD : {}, Train Acc : {}, Valid Acc : {} , Train Loss : {}, Valid Loss : {}".format(epoch +1, lr, wd, 
                                                                                                                       best_ta, best_va, 
                                                                                                                       best_tl, best_vl))
        iter_ += 1  

Epoch 500 : 100%|=| 500/500 [21:23<00:00,  2.57s/it, LR=0.0001, WD=0.001, TA=0.9


Epoch : 500, LR : 0.0001, WD : 0.001, Train Acc : 0.8675757575757576, Valid Acc : 0.808252427184466 , Train Loss : 0.3158948878542735, Valid Loss : 0.4119804914181049


Epoch 500 : 100%|=| 500/500 [21:22<00:00,  2.56s/it, LR=0.0001, WD=0.0001, TA=0.


Epoch : 500, LR : 0.0001, WD : 0.0001, Train Acc : 0.8451515151515151, Valid Acc : 0.8106796116504854 , Train Loss : 0.3503010215667578, Valid Loss : 0.42245798386060274


Epoch 500 : 100%|=| 500/500 [21:19<00:00,  2.56s/it, LR=0.0001, WD=1e-5, TA=0.99


...

Epoch 500 : 100%|=| 500/500 [21:32<00:00,  2.59s/it, LR=1e-5, WD=1e-5, TA=0.908,


Epoch : 500, LR : 1e-05, WD : 1e-05, Train Acc : 0.863030303030303, Valid Acc : 0.8131067961165048 , Train Loss : 0.3255628916220023, Valid Loss : 0.4225354492664337


Epoch 500 : 100%|=| 500/500 [21:32<00:00,  2.59s/it, LR=1e-6, WD=0.001, TA=0.808


Epoch : 500, LR : 1e-06, WD : 0.001, Train Acc : 0.8051515151515152, Valid Acc : 0.7961165048543689 , Train Loss : 0.42466455397124475, Valid Loss : 0.4662620723247528


Epoch 479 :  96%| | 478/500 [19:26<00:51,  2.35s/it, LR=1e-6, WD=0.0001, TA=0.80IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
def inference(loader, lr, wd):
    y_pred = list()
    y_test = list()
    best_model = torch.load("./checkpoint/grid_search/0_binary_depression_unimodal/GAT/best_model_binary_{}_{}.pt".format(lr, wd))
    best_model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        data.y = data.y.to(device)
        
        out = best_model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        
        y_pred.extend(pred.tolist())
        y_test.extend(data.y.tolist())
        
    return y_test, y_pred

In [16]:
target_names = ['daily', 'depression']

In [17]:
#epoch 500
for lr in LR_LIST :
    for wd in WEIGHT_DECAY :
        print(lr, wd)
        y_test, y_pred = inference(test_loader, lr, wd)
        print(classification_report(y_test, y_pred, target_names = target_names, digits = 4))

0.0001 0.001
              precision    recall  f1-score   support

       daily     0.8040    0.8040    0.8040       199
  depression     0.8178    0.8178    0.8178       214

    accuracy                         0.8111       413
   macro avg     0.8109    0.8109    0.8109       413
weighted avg     0.8111    0.8111    0.8111       413

0.0001 0.0001
              precision    recall  f1-score   support

       daily     0.7860    0.8492    0.8164       199
  depression     0.8485    0.7850    0.8155       214

    accuracy                         0.8160       413
   macro avg     0.8173    0.8171    0.8160       413
weighted avg     0.8184    0.8160    0.8160       413

0.0001 1e-05
              precision    recall  f1-score   support

       daily     0.7981    0.8543    0.8252       199
  depression     0.8550    0.7991    0.8261       214

    accuracy                         0.8257       413
   macro avg     0.8266    0.8267    0.8257       413
weighted avg     0.8276    0.8257 